In [1]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import time
import pandas as pd

In [3]:
joblinks= pd.read_csv(r'C:\Users\aeshna_gupta\OneDrive - Dell Technologies\DGA\Udemy courses\Web scrapping\link_by_areas.csv')
joblinks.head()

,type,link
0,Accounting Jobs,https://www.naukri.com/accounting-jobs?xt=cats...
1,Interior Design Jobs,https://www.naukri.com/interior-design-jobs?xt...
2,Bank Jobs,https://www.naukri.com/bank-jobs?xt=catsrch&qf...
3,Content Writing Jobs,https://www.naukri.com/content-writing-jobs?xt...
4,Consultant Jobs,https://www.naukri.com/consultant-jobs?xt=cats...


Convert url to list

In [ ]:
urls= joblinks['link'].tolist()

segment the parts of url

In [7]:
! pip install yarl

Defaulting to user installation because normal site-packages is not writeable
     -------------------------------------- 55.9/55.9 KB 415.4 kB/s eta 0:00:00


You should consider upgrading via the 'C:\Program Files\Python310\python.exe -m pip install --upgrade pip' command.


In [8]:
import yarl
from yarl import URL

n=[]

for i in urls:
    n.append(URL(i).path) # get name of jobs from url


In [10]:
m = []

for i in urls:
    m.append(URL(i).query_string) # the last part of the url 'xt=catsrch&qf[]=1'

In [12]:
# concat all parts together
gen_url=[]

for i in range(len(n)):
    url = 'https://www.naukri.com' + n[i] + '-{}?' + m[i]  # need placeholder for page number 
    gen_url.append(url)
    

In [14]:
gen_url[0]

'https://www.naukri.com/accounting-jobs-{}?xt=catsrch&qf[]=1'

Scraping

In [56]:
df = pd.DataFrame(columns=['Job_Title','Company','Scraping_Date','Salary','Location','Tags_Associated','Posting_Date'])

In [57]:
# ! pip install html5lib

In [ ]:
for page in range(1,2):
    for urll in gen_url[0:5]:
        url = urll.format(page)
        
        driver = webdriver.Chrome(ChromeDriverManager().install())
        driver.get(url)
        time.sleep(5)

        soup = BeautifulSoup(driver.page_source,"html.parser") # to extract information from page
        driver.close()

        # print(soup.prettify())
        results = soup.find(class_='list')
        job_elems = results.find_all('article',class_='jobTuple bgWhite br4 mb-8')

        for job_elem in job_elems:
            Job_titles = job_elem.find('a',class_='title fw500 ellipsis')
            print(Job_titles.text)

        # Company
            Company = job_elem.find('a',class_='subTitle ellipsis fleft')
            print(Company.text)
        # Date Scraped
            from datetime import date
            today = date.today()
                    # dd/mm/YY
            date_today = today.strftime("%d/%m/%Y")
            print(date_today)          # Date when we are scraping 
        # Salary
            Sal = job_elem.find('li',class_='fleft grey-text br2 placeHolderLi salary')
            Sal_span = Sal.find('span',class_='ellipsis fleft fs12 lh16')
            if Sal_span is None:
                continue
            else:
                Salary = Sal_span.text
                print(Salary)

            
        # Location for the job post
            Loc = job_elem.find('li',class_='fleft grey-text br2 placeHolderLi location')
            Loc_exp = Loc.find('span',class_='ellipsis fleft fs12 lh16 locWdth')
            if Loc_exp is None:
                continue
            else:
                Location = Loc_exp.text
                print(Location)
         #tags
     
            tags = job_elem.find('li',class_='fleft fs12 grey-text lh16 dot')

            if tags is None:
                continue
            else:
                assoc_tags = tags.text
                print(assoc_tags)

            # Date job Posted
            date = job_elem.find("div",["type br2 fleft grey","type br2 fleft green"])
            date_posted = date.find('span',class_='fleft fw500')
            if date_posted is None:
                continue
            else:
                Posting_Date = date_posted.text
                print(Posting_Date)

           
           
           
            df=df.append({'Job_Title':Job_titles.text,'Company':Company.text,'Scraping_Date':date_today, 'Salary':Salary,'Location':Location,'Tags_Associated':assoc_tags,'Posting_Date':Posting_Date},ignore_index = True)
                  
        

In [59]:
df.head()

,Job_Title,Company,Scraping_Date,Salary,Location,Tags_Associated,Posting_Date
0,Associate/ Senior Associate in UK Accounts/ T...,Anderson Business Solutions,04/11/2022,"2,25,000 - 4,50,000 PA.","Thane, Navi Mumbai, Mumbai (All Areas)(Ghodbun...",Accounting,4 Days Ago
1,HCL || Chartered Accountant-FP&A,HCLTech,04/11/2022,"7,00,000 - 10,00,000 PA.","Noida, Delhi / NCR",forecasting,Just Now
2,HLCL || Chartered Accountant-Controllership,HCLTech,04/11/2022,"7,00,000 - 10,00,000 PA.","Noida, Delhi / NCR",qualified chartered accountant,Just Now
3,B.Com (Full Time) Fresher from Delhi Universit...,Shahi,04/11/2022,"3,00,000 - 4,75,000 PA.",Faridabad,bcom,4 Days Ago
4,Accounts executive-MBA Fresher,Shahi,04/11/2022,Not disclosed,Faridabad,Finance,4 Days Ago


In [60]:
df.shape

(86, 7)

We have successfully rendered Job details from the first 5 pages of Naukri site.